In [11]:
source_url,test_name,grade,sentence, LFT=[],[],[],[],[]
total_links = []

In [12]:
import pandas as pd
import numpy as np

def load_csv(csv_path):
    global source_url,test_name,grade,sentence, LFT
    datas = pd.read_csv(csv_path)
    datas_array = np.array(datas)
    source_url += datas_array[:,1].tolist()
    test_name += datas_array[:,2].tolist()
    grade += datas_array[:,3].tolist()
    LFT += datas_array[:,4].tolist()
    sentence += datas_array[:,5].tolist()

load_csv('data.csv')
load_csv('data233.csv')

In [14]:
total_links = list(set(source_url))

In [28]:
len(source_url)

38278

In [74]:
from selenium import webdriver
driver = webdriver.Chrome()

In [76]:
driver.get('https://quizizz.com/admin/quiz/5f589da2cf5b97001b513da7/logical-fallacies-practice')

In [81]:
def open_answer(driver):
    answer_button = driver.find_element_by_class_name('show-answers-btn')
    button_status = answer_button.find_element_by_class_name('far').get_attribute('class')
    if button_status == 'far fa-eye':
        answer_button.click()
open_answer(driver)

In [78]:
def get_single_driver(driver):
    open_answer(driver)
        
    questions = driver.find_elements_by_class_name('question')
    answer = []
    for question in questions:
        text = [i.text for i in question.find_elements_by_class_name('query')]
        foptions = question.find_elements_by_class_name('text-only-option')
        foptions = [i.text for i in foptions if i.find_element_by_class_name('option-marker').get_attribute('class') == 'option-marker true']
        if (len(foptions)==0):
            foptions = [i.text for i in question.find_elements_by_class_name('text')]
        if len(text)==1 and len(foptions)>=1:
            answer.append([text[0],' & '.join(foptions)])
    
    return answer
    
get_single_driver(driver)

<selenium.webdriver.remote.webelement.WebElement (session="b5b04e78c28c6775582289370e6a547d", element="ef01f861-0d58-48bd-b13e-73a7bf95a654")>
far fa-eye-slash


[['Q. Identify the logical fallacy committed.\n\nIt is ridiculous to have spent thousands of dollars to rescue those two whales trapped in the Arctic ice. Why, look at all the people trapped in jobs they don’t like.',
  'Weak analogy'],
 ['Q. Identify the logical fallacy committed.\n\nPlagiarism is deceitful because it is dishonest.',
  'Begging the question'],
 ["Q. Identify the logical fallacy committed.\n\nWater fluoridation affects the brain. Citywide, student’s test scores began to drop five months after fluoridation began. The water fluoridation must have caused the student's test scores to drop.",
  'Post hoc'],
 ['Q. Identify the logical fallacy committed.\n\nI know three redheads who have terrible tempers, and since Annabel has red hair, I’ll bet she has a terrible temper too.',
  'Hasty generalization'],
 ['Q. Identify the logical fallacy committed.\n\nYou support capital punishment just because you want an “eye for an eye,” but I have several good reasons to believe that cap

In [31]:
driver.get('https://quizizz.com/admin/search/logical%20fallacies?sortBy=_score&grade=all&subject=All&langs=English&numQuestions=&duplicates=false&studentQuizzes=false&safeSearch=true&type=quiz,')

In [53]:
results=driver.find_element_by_class_name('search-results').find_elements_by_class_name('router-link')
len(results)

504

In [54]:
# Extract the valid link
links = [i.get_attribute('href')for i in results]
def link_filter(link):
    return ('https://quizizz.com/admin/' in link )
links = [link for link in links if link_filter(link)]

In [55]:
# Remove duplicate links
links = [i for i in links if not i in total_links]
for link in links:
    total_links.append(link)
len(links)

203

In [58]:
len(total_links)
total_links_old = list(set(source_url))

In [82]:
llinks = [i for i in total_links if not i in total_links_old]
len(llinks)

932

In [83]:
li, grades, names, ans = [], [], [], []

In [ ]:
import time

li, grades, names, ans = [], [], [], []
qwq = 0
for link in llinks:
    print(qwq)
    qwq+=1
    driver.get(link)
    time.sleep(2)
    new_ans = get_single_driver(driver)
    ans.append(new_ans)
    li.append([link]*len(new_ans))
    grades.append(driver.find_elements_by_class_name('grades')[0].text)
    names.append(driver.find_elements_by_class_name('quiz-name-text')[0].text)
print(len(ans))

0
1
2
3
4
5
6


In [ ]:
# source_url,test_name,grade,sentence, LFT=[],[],[],[],[]
for i in range(556):
    for j in ans[i]:
        source_url.append(llinks[i])
        test_name.append(names[i])
        grade.append(grades[i])
        sentence.append(j[0])
        LFT.append(j[1])

In [ ]:
import pandas as pd
datas = pd.DataFrame({'source_url':source_url,
                      'test_name':test_name,
                      'grade':grade,
                      'Logical Fallacy Types':LFT,
                      'sentence':sentence
                    })
datas.to_csv('data.csv')